In [ ]:
# import sys
# !{sys.executable} -m pip install numpy pandas matplotlib sklearn seaborn
# !{sys.executable} -m pip install --upgrade gensim
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install beautifulsoup4
# import nltk
# nltk.download('punkt')

In [1]:
import os
import re
import gensim
import multiprocessing
import random
import logging
import numpy as np
import zipfile

from urllib import request

from pathlib import Path
from os import listdir
from os.path import isfile, join
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup

from sklearn.cluster import KMeans
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
SKIP_FILES = ""
NEWLINE = '\n'

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

"""
read training files
"""
def read_files(path):
    print("path: {}...".format(path))
    for root, dirnames, filenames in os.walk(path):
        for dirname in dirnames:
            read_files(os.path.join(root, dirname))
        for filename in filenames:
            if filename not in SKIP_FILES:
                filepath = os.path.join(root, filename)
                if os.path.isfile(filepath):
                    lines = []
                    f = open(filepath, encoding='latin-1')
                    for line in f:
                        lines.append(line)
                    f.close()
                    content = NEWLINE.join(lines)
                    yield filename, content

def download(url, filename):
    """Download a file if not present"""
    if not os.path.exists(filename):
        print("downloading {}...".format(filename))
        filename, _ = request.urlretrieve(url + filename, filename)

        print("extracting {}...".format(filename))
        with zipfile.ZipFile(filename) as f:
            f.extractall()
            print("extracting {} done".format(filename))

    """directory data"""
    data_path = filename.replace('.zip','')
    if not os.path.exists(data_path):
        print("extracting {}...".format(filename))
        with zipfile.ZipFile(filename) as f:
            f.extractall()
            print("extracting {} done".format(filename))

    """directory model for saving model while training"""
    if not os.path.exists('model'):
        os.mkdir('model')
        print("directory model created...")

    return data_path

In [3]:
def get_data(url, filename):
    data_path = download(url, filename)
    documents = []
    print("building documents...")
    for fname, text in read_files(data_path):
        documents.append(clean_str(text).split(' '))
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]
    print("building documents done")
    return documents

In [4]:
# download data training
documents = get_data(url='https://github.com/kadriansyah/notebook/raw/master/alodokter-doc2vec-article/', filename="data.zip")
print("we have {} documents".format(len(documents)))

downloading data.zip...
extracting data.zip...
directory model created...
building documents...
path: data...
building documents done
we have 7932 documents


In [5]:
documents[0]

TaggedDocument(words=['penyebab', 'bayi', 'muntah', 'setelah', 'minum', 'asi', 'dan', 'cara', 'mengatasinya', 'bayi', 'muntah', 'setelah', 'minum', 'asi', '(', 'air', 'susu', 'ibu', ')', 'adalah', 'keluhan', 'yang', 'sering', 'terjadi', 'sebagian', 'bayi', 'bahkan', 'mengalaminya', 'hampir', 'setiap', 'kali', 'selesai', 'menyusu', 'meski', 'umumnya', 'normal', 'kondisi', 'ini', 'bisa', 'juga', 'disebabkan', 'oleh', 'gangguan', 'berbahaya', 'yang', 'harus', 'diwaspadai', 'bayi', 'muntah', 'setelah', 'minum', 'asi', 'dikenal', 'dengan', 'istilah', 'gumoh', 'gumoh', 'dikatakan', 'normal', 'apabila', 'tidak', 'menyebabkan', 'bayi', 'rewel', 'atau', 'sesak', 'napas', 'meskipun', 'dapat', 'dicegah', 'kondisi', 'tersebut', 'tidak', 'memerlukan', 'penanganan', 'khusus', 'dan', 'normal', 'terjadi', 'penyebab', 'bayi', 'muntah', 'setelah', 'minum', 'asi', 'gumoh', 'disebabkan', 'oleh', 'asi', 'atau', 'susu', 'yang', 'ditelan', 'bayi', 'kembali', 'ke', 'kerongkongan', 'karena', 'otot', 'di', 'sal

In [6]:
def evaluate(model, documents, steps):
    percentiles = np.zeros(steps)
    for step in range(steps):
        docid = np.random.randint(model.docvecs.count)
        inferred_vector = model.infer_vector(documents[docid][0])
        similars = model.docvecs.most_similar(positive=[inferred_vector], topn=10)
        for idx,simdoc in enumerate(similars):
            if simdoc[0] == docid:
                print("found similar document with id {} in position {} with similarity score {}".format(simdoc[0], idx, simdoc[1]))
                percentiles[step] = ((len(similars) - idx) / len(similars)) * 100
                break
    return np.mean(percentiles)

def train(documents=documents, model_name="model/alodokter-articles-doc2vec.model", max_epochs=50, patience=3):
    best_mean_percentiles = 0
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = Doc2Vec(dm=1, vector_size=300, window=2, alpha=0.1, min_alpha=0.0001, min_count=5, epochs=1, workers=5)
    model.build_vocab(documents)
    for epoch in range(max_epochs):
        print('training epoch {:d} ...'.format(epoch))
        model.train(documents, total_examples=model.corpus_count,epochs=model.epochs)
        mean_percentiles = evaluate(model,documents,10)
        print('mean percentiles: {:.2f}'.format(mean_percentiles))
        
        if mean_percentiles < best_mean_percentiles:
            print("current mean_percentiles: {:.2f}, best: {:.2f}".format(mean_percentiles, best_mean_percentiles))
            patience = patience-1
        else:
            best_mean_percentiles = mean_percentiles
            print("========== Saving best model with mean_percentiles: {:.2f} ==========".format(mean_percentiles))
            model.save(model_name)
            patience = patience+1
        
        if patience == 0:
            print("early stop...")
            print("========== Saving best model with mean_percentiles: {:.2f} ==========".format(best_mean_percentiles))
            break
    
    return model

In [7]:
model = train(documents)

2020-02-23 09:59:53,541 : INFO : collecting all words and their counts
2020-02-23 09:59:53,542 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-02-23 09:59:54,211 : INFO : collected 54008 word types and 7932 unique tags from a corpus of 7932 examples and 4866436 words
2020-02-23 09:59:54,212 : INFO : Loading a fresh vocabulary
2020-02-23 09:59:54,248 : INFO : effective_min_count=5 retains 15409 unique words (28% of original 54008, drops 38599)
2020-02-23 09:59:54,248 : INFO : effective_min_count=5 leaves 4811064 word corpus (98% of original 4866436, drops 55372)
2020-02-23 09:59:54,290 : INFO : deleting the raw counts dictionary of 54008 items
2020-02-23 09:59:54,292 : INFO : sample=0.001 downsamples 51 most-common words
2020-02-23 09:59:54,293 : INFO : downsampling leaves estimated 4105293 word corpus (85.3% of prior 4811064)
2020-02-23 09:59:54,333 : INFO : estimated required memory for 15409 words and 300 dimensions: 54204500 bytes
2020-02-23 09:5

training epoch 0 ...


2020-02-23 09:59:58,943 : INFO : EPOCH 1 - PROGRESS: at 36.91% examples, 1515329 words/s, in_qsize 9, out_qsize 0
2020-02-23 09:59:59,947 : INFO : EPOCH 1 - PROGRESS: at 75.26% examples, 1540463 words/s, in_qsize 9, out_qsize 0
2020-02-23 10:00:00,621 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-02-23 10:00:00,629 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-02-23 10:00:00,631 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-23 10:00:00,631 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-23 10:00:00,632 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-23 10:00:00,633 : INFO : EPOCH - 1 : training on 4866436 raw words (4113427 effective words) took 2.7s, 1525990 effective words/s
2020-02-23 10:00:00,634 : INFO : training on a 4866436 raw words (4113427 effective words) took 2.7s, 1524820 effective words/s
2020-02-23 10:00:00,636 : INFO : precomputing L2-no

found similar document with id 3700 in position 0 with similarity score 0.6231238842010498
found similar document with id 7568 in position 0 with similarity score 0.7852587103843689
found similar document with id 5502 in position 0 with similarity score 0.7272913455963135
found similar document with id 4699 in position 0 with similarity score 0.750701367855072
found similar document with id 4810 in position 0 with similarity score 0.7558729648590088
found similar document with id 4295 in position 0 with similarity score 0.8224493265151978
found similar document with id 6448 in position 0 with similarity score 0.8298599720001221
found similar document with id 2632 in position 0 with similarity score 0.7019071578979492
found similar document with id 3084 in position 0 with similarity score 0.585481584072113
mean percentiles: 90.00
========== Saving best model with mean_percentiles: 90.00 ==========


2020-02-23 10:00:01,124 : INFO : saved model/alodokter-articles-doc2vec.model
2020-02-23 10:00:01,125 : WARNING : Effective 'alpha' higher than previous training cycles
2020-02-23 10:00:01,125 : INFO : training model with 5 workers on 15409 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2


...

found similar document with id 4119 in position 0 with similarity score 0.4817700982093811
found similar document with id 2976 in position 3 with similarity score 0.4866436719894409
found similar document with id 6770 in position 0 with similarity score 0.5098720788955688
found similar document with id 7777 in position 0 with similarity score 0.5361309051513672
found similar document with id 4517 in position 1 with similarity score 0.5081150531768799
found similar document with id 4165 in position 0 with similarity score 0.5586714148521423
found similar document with id 5589 in position 0 with similarity score 0.5203511118888855
found similar document with id 1063 in position 0 with similarity score 0.5195136070251465
mean percentiles: 76.00
current mean_percentiles: 76.00, best: 100.00
training epoch 11 ...


2020-02-23 10:00:32,061 : INFO : EPOCH 1 - PROGRESS: at 37.32% examples, 1532816 words/s, in_qsize 10, out_qsize 0
2020-02-23 10:00:33,077 : INFO : EPOCH 1 - PROGRESS: at 76.07% examples, 1548092 words/s, in_qsize 10, out_qsize 1
2020-02-23 10:00:33,662 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-02-23 10:00:33,667 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-02-23 10:00:33,668 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-23 10:00:33,671 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-23 10:00:33,672 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-23 10:00:33,672 : INFO : EPOCH - 1 : training on 4866436 raw words (4113151 effective words) took 2.6s, 1572415 effective words/s
2020-02-23 10:00:33,673 : INFO : training on a 4866436 raw words (4113151 effective words) took 2.6s, 1570482 effective words/s
2020-02-23 10:00:33,697 : WARNING : Effective 'al

found similar document with id 6504 in position 0 with similarity score 0.43613314628601074
found similar document with id 7434 in position 0 with similarity score 0.5340348482131958
found similar document with id 531 in position 8 with similarity score 0.48238855600357056
found similar document with id 7359 in position 1 with similarity score 0.3738921582698822
found similar document with id 7758 in position 5 with similarity score 0.37554943561553955
found similar document with id 6593 in position 0 with similarity score 0.3600296080112457
mean percentiles: 46.00
current mean_percentiles: 46.00, best: 100.00
training epoch 12 ...


2020-02-23 10:00:34,704 : INFO : EPOCH 1 - PROGRESS: at 36.28% examples, 1486485 words/s, in_qsize 9, out_qsize 0
2020-02-23 10:00:35,715 : INFO : EPOCH 1 - PROGRESS: at 74.47% examples, 1521212 words/s, in_qsize 9, out_qsize 0
2020-02-23 10:00:36,375 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-02-23 10:00:36,380 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-02-23 10:00:36,382 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-23 10:00:36,382 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-23 10:00:36,384 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-23 10:00:36,385 : INFO : EPOCH - 1 : training on 4866436 raw words (4114068 effective words) took 2.7s, 1532675 effective words/s
2020-02-23 10:00:36,385 : INFO : training on a 4866436 raw words (4114068 effective words) took 2.7s, 1530995 effective words/s


found similar document with id 5518 in position 0 with similarity score 0.33522993326187134
found similar document with id 3107 in position 3 with similarity score 0.45306405425071716
found similar document with id 6105 in position 1 with similarity score 0.4211587607860565
mean percentiles: 26.00
current mean_percentiles: 26.00, best: 100.00
early stop...
========== Saving best model with mean_percentiles: 100.00 ==========
